In [33]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests as rq
from scipy import stats
import math
from secret_case import API_TOKEN

In [34]:
stocks = pd.read_csv('/Users/hashimkhan/Desktop/algo_trading_final/sp_500_stocks.csv')

In [35]:
#Function sourced from: https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def chunks (lst, n):
    """Yield succisve n-sized chunks form lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [36]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

dataframes = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data = rq.get(batch_api_url, headers=headers, params=querystring).json()

    rows = []
    for result in data.get('quoteResponse', {}).get('result', []):
        
        symbol = result.get('symbol', np.nan)
        
        regular_market_price = float(result.get('regularMarketPrice', np.nan))

        trailing_pe = float(result.get('trailingPE', np.nan))


        
        rows.append([symbol, regular_market_price, trailing_pe, np.nan])

    dataframe = pd.DataFrame(rows, columns=my_columns)
    dataframes.append(dataframe)

final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,112.4050,29.425392,NaN
1,AAL,12.9700,3.395288,NaN
2,AAP,54.7500,9.455958,NaN
3,AAPL,170.6600,28.586266,NaN
4,ABBV,151.6050,31.194443,NaN
...,...,...,...,...
499,YUM,123.7150,24.992930,NaN
500,ZBH,112.0600,46.691666,NaN
501,ZBRA,236.4775,19.335855,NaN
502,ZION,33.9400,5.985890,NaN


In [37]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0 ]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)

In [39]:
final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,LVS,45.495,649.92850,NaN
1,TAP,62.540,329.15790,NaN
2,WELL,80.510,309.65387,NaN
3,VTR,42.120,191.45454,NaN
4,STE,222.120,182.06557,NaN


In [40]:
def portfolio_input():
    global portfolio_size
    portfolio_size = float(input('Enter the size of your portfolio:'))

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That is not a number! \nPlease try again:")
        portfolio_size = float(input('Enter the size of your portfolio:'))

In [41]:
portfolio_input()

In [43]:
position_size = float(portfolio_size) / len(final_dataframe.index)

In [50]:
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])

final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,LVS,45.495,649.92850,43960.0
1,TAP,62.540,329.15790,31979.0
2,WELL,80.510,309.65387,24841.0
3,VTR,42.120,191.45454,47483.0
4,STE,222.120,182.06557,9004.0
